<a href="https://colab.research.google.com/github/mdsozon24/Fake-news-detection-web-scraping-/blob/main/Fake_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fake_useragent

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import re
from fake_useragent import UserAgent
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import time

# Download NLTK stopwords and punkt
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
urls = [
    "https://www.jugantor.com/","https://www.prothomalo.com/","https://www.bd-pratidin.com/","https://www.ittefaq.com.bd/",
    "https://www.kalerkantho.com/","https://dailynayadiganta.com/","https://www.amarsangbad.com/","https://www.protidinersangbad.com/",
    "https://dailysangram.com/","https://mzamin.com/","https://dainikamadershomoy.com/","https://www.bonikbarta.com/",
    "https://samakal.com/","https://dailyjanakantha.com/","https://www.jaijaidinbd.com/","https://bhorerkagoj.com/",
    "https://dailyinqilab.com/","https://sangbad.net.bd/","https://suprobhat.com/","https://www.bd-journal.com/","https://www.alokitobangladesh.com/",
    "https://www.ajkerbazzar.com/","https://amaderorthoneeti.com/","https://bangladeshpost.net/","https://sorejominbarta.com/",
    "https://khoborpatrabd.com/","https://www.dailyvorerpata.com/","https://shomoyeralo.com/","https://www.kalbela.com/","https://sharebiz.net/",
    "https://www.kholakagojbd.com/","https://gonokantho.com/","https://www.thedailystar.net/","https://observerbd.com/",
    "https://thefinancialexpress.com.bd/","https://www.deshrupantor.com/","https://www.bangladesherkhabor.net/","https://bd-bulletin.com/",
    "https://www.tbsnews.net/","https://www.dhakatribune.com/","https://www.ajkerpatrika.com/","https://www.dainikbangla.com.bd/",
    "https://bangla.bdnews24.com/",
    "https://www.banglanews24.com/",
    "https://www.jagonews24.com/",
    "https://www.priyo.com/",
    "https://dainikamadershomoy.com/",
    "https://www.bd24live.com/",
    "https://www.dhakapost.com/",
    "https://www.banglatribune.com/",
    "https://www.bdnews24.com/",
    "https://unb.com.bd/",
    "https://amar-desh24.com/",
    "https://sarabangla.net/",
    "https://thedhakatimes.com/",
    "https://dmpnews.org/",
    "https://www.thereport24.com/",
    "https://somoyerkonthosor.com/",
    "https://sharenews24.com/",
    "https://sunbd24.com/",
    "https://orthosongbad.com/",
    "https://ajkersottasangbad24.com/",
    "https://thedhakapost.com/",
    "https://www.onenewsbd.com/",
    "https://deshnews.net/",
    "https://deshersangbad.com/",
    "https://lastnewsbd.com/",
    "https://www.bangladaily24.com/",
    "https://www.protikhon.com/",
    "https://bartabangla.com/",
    "https://bdtype.com/",
    "https://inews.zoombangla.com/",
    "https://khoborprotidin24.com/",
    "https://bdcrictime.com/",
    "https://www.bbc24news.com/",
    "https://www.latestbdnews.com/",
    "https://bartabazar.com/",
    "https://www.dhakaprotidin.com/",
    "https://eibela.com/",
    "https://techshohor.com/",
    "https://techzoom.tv/",
    "https://www.channelionline.com/","https://www.somoynews.tv/","https://www.ntvbd.com/","channel24bd.tv",
    "https://rtvonline.com/","https://ekushey-tv.com/","https://jamuna.tv/","satv.tv","desh.tv","https://maasranga.tv/",
    "https://www.bvnews24.com/","https://ekattor.tv/","https://www.atnnewstv.com/","https://mytvbd.tv/",
    "https://boishakhionline.com/","https://mohona.tv/","https://gazitv.com/","https://btv.gov.bd/",
    "https://gaanbangla.tv/lander?oref=https%3A%2F%2Fwww.allbanglanewspaper.xyz%2F","https://dainikazadi.net/",
    "https://dainikpurbokone.net/","https://purbanchal.com/","https://www.ukhiyanews.com/","https://rajshahinews24.com/",
    "https://chattogramnews.com/","https://dailykhowai.com/","https://karatoa.com.bd/","https://dainiksylhet.com/",
    "https://euttorpurbo.com/paper/index/2025-09-29","https://www.amaderbarisal.com/","https://surmatimes.com/",
    "https://dailysylhet.com/","https://chandpurtimes.com/","https://bograsangbad.com/","https://rajshahirsomoy.com/",
    "https://sylhetexpress.net/","https://lakshmipur24.com/","https://chakarianews.com/","https://gramerkagoj.com/",
    "https://khulnanchal.com/","https://amarnoakhali.com/","https://www.anandabazar.com/","https://www.aajkaal.in/",
    "https://kolkata24x7.in/","https://zeenews.india.com/bengali/","https://bengali.abplive.com/","https://www.ndtv.com/",
    "https://syandanpatrika.com/","https://www.sangbadpratidin.in/","https://bartamanpatrika.com/","https://ganashakti.com/",
    "https://eisamay.com/","https://uttarbangasambad.com/","https://www.anandabazar.com/","https://dainiksambad.net/",
    "https://www.aamadermalda.in/","https://bangla.ganashakti.co.in/","https://bangladesherkhela.com/",
    "https://weeklybangalee.com/","https://dhakacourier.com.bd/","https://www.bichitra.org/","https://www.weeklyholiday.net/",
    "https://www.kaliokalam.com/","https://scientificbangladesh.com/","https://bdjobs.com/","https://bdjobstoday.com/",
    "https://job.com.bd/","https://www.workforceaustralia.gov.au/","https://educationbangla.com/","https://www.nu.ac.bd/"
       ]

In [ ]:
session = requests.Session()

In [ ]:
df_0 = []

In [ ]:
ua = UserAgent()

retry_strategy = Retry(
    total=3,  # Maximum number of retries
    backoff_factor=1,  # Exponential backoff factor (1s, 2s, 4s delays)
    status_forcelist=[429, 500, 502, 503, 504],  # HTTP statuses to retry on
    allowed_methods=["HEAD", "GET", "OPTIONS"] # HTTP methods to retry
)

In [ ]:
adapter = HTTPAdapter(max_retries=retry_strategy)
session.mount("http://", adapter)
session.mount("https://", adapter)

In [ ]:
for url in urls:
    try:
        headers = {'User-Agent': ua.random, 'Accept-Language': 'en,bn'} # Generate and add a random user agent and accept language
        r = session.get(url, timeout=10, headers=headers)
        r.raise_for_status()

        soup = BeautifulSoup(r.text, 'html.parser')
        data = soup.get_text()

        for line in data.splitlines():
            cleaned_line = re.sub(r'\s+', ' ', line.strip())
            if cleaned_line and len(cleaned_line) > 20:
                df_0.append(cleaned_line)

    except requests.exceptions.RequestException as e:
        print(f"Error fetching {url}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred while processing {url}: {e}")

    time.sleep(2)

Error fetching https://www.kalerkantho.com/: 403 Client Error: Forbidden for url: https://www.kalerkantho.com/
Error fetching https://www.protidinersangbad.com/: 403 Client Error: Forbidden for url: https://www.protidinersangbad.com/
Error fetching https://dailyjanakantha.com/: 403 Client Error: Forbidden for url: https://dailyjanakantha.com/
Error fetching https://www.jaijaidinbd.com/: 403 Client Error: Forbidden for url: https://www.jaijaidinbd.com/
Error fetching https://www.bd-journal.com/: 403 Client Error: Forbidden for url: https://www.bd-journal.com/
Error fetching https://khoborprotidin24.com/: 403 Client Error: Forbidden for url: https://khoborprotidin24.com/
Error fetching https://techshohor.com/: 403 Client Error: Forbidden for url: https://techshohor.com/
Error fetching https://www.somoynews.tv/: 403 Client Error: Forbidden for url: https://www.somoynews.tv/
Error fetching channel24bd.tv: Invalid URL 'channel24bd.tv': No scheme supplied. Perhaps you meant https://channel24

Error fetching https://rajshahinews24.com/: HTTPSConnectionPool(host='rajshahinews24.com', port=443): Max retries exceeded with url: / (Caused by ReadTimeoutError("HTTPSConnectionPool(host='rajshahinews24.com', port=443): Read timed out. (read timeout=10)"))
Error fetching https://www.ndtv.com/: 403 Client Error: Forbidden for url: https://www.ndtv.com/


In [ ]:
df_0

['Jugantor: Bangla Newspaper | Bangla News | ব্রেকিং নিউজ',
 'স্থানীয় সরকার বিশেষজ্ঞ তোফায়েল আহমেদ আর নেই',
 'বুধবার (৮ অক্টোবর) সন্ধ্যা সাড়ে ৭টার দিকে রাজধানীর একটি হাসপাতালে তিনি শেষ নিশ্বাস ত্যাগ করেন বলে জানিয়েছেন ...',
 '০৮ অক্টোবর ২০২৫, ১১:২৪ পিএম',
 'জিয়াউর রহমানের মাজারে কুরআন তেলাওয়াত ও দোয়া করলেন খালেদা জিয়া',
 'ওমানে সড়ক দুর্ঘটনায় ৭ বাংলাদেশি নিহত',
 'জিয়াউর রহমানের মাজারে যাচ্ছেন খালেদা জিয়া',
 '৬৭ রানে ৭ উইকেট পতনে হয়নি চ্যালেঞ্জিং স্কোর',
 '‘সেফ এক্সিট’ ইস্যুতে রাজনৈতিক অঙ্গনে উত্তাপ',
 'ভারতের পররাষ্ট্রসচিবের বক্তব্য ‘অযৌক্তিক’: তৌহিদ হোসেন',
 'স্বর্ণের দামে আবারও রেকর্ড, ভরি কত?',
 'ইতিহাসে সর্বোচ্চ দামে স্বর্ণ, বাড়ছে যেসব কারণে',
 'আল-আকসার মালিক এখন আমরা’—দখলদার ইসরাইলি মন্ত্রীর উসকানিমূলক ঘোষণা',
 'বোরো মৌসুমে ডিলার নিয়োগ নীতিমালা করলে বিশৃঙ্খলা সৃষ্টির শঙ্কা',
 'সার ডিলার নিয়োগ ও বিতরণসংক্রান্ত সমন্বিত নীতিমালা করছে সরকার। কিন্তু আসন্ন বোরো মৌসুমে এ নীতিমালা বাস্তবায়নে সরকারের সিদ্ধান্ত ...',
 'স্থানীয় সরকার বিশেষজ্ঞ তোফায়েল আহমেদ আর নেই',
 'আইনশৃঙ্খলা বাহিনীর নির্য

In [ ]:
df_0 = pd.DataFrame(df_0, columns=['Text'])
df_0

,Text
0,Jugantor: Bangla Newspaper | Bangla News | ব্র...
1,স্থানীয় সরকার বিশেষজ্ঞ তোফায়েল আহমেদ আর নেই
2,বুধবার (৮ অক্টোবর) সন্ধ্যা সাড়ে ৭টার দিকে রাজধ...
3,"০৮ অক্টোবর ২০২৫, ১১:২৪ পিএম"
4,জিয়াউর রহমানের মাজারে কুরআন তেলাওয়াত ও দোয়া কর...
...,...
18003,"National University , Bangladesh"
18004,Email - registrar@nu.ac.bd
18005,Emmergency - +8801734790892
18006,Web:www.nu.ac.bd | E-mail: registrar@nu.ac.bd


In [ ]:
bangla_dataset = pd.read_csv('/content/drive/MyDrive/Datasets/BanglaNews8.csv')

In [ ]:
df_1 = bangla_dataset["Text"].head(3000)
df_1

,Text
0,গ্যাটকো দুর্নীতি মামলায় বিএনপি চেয়ারপারসন খালে...
1,সৈয়দ এম ওমর তৈয়ব প্রাইম ব্যাংক উপব্যবস্থাপনা প...
2,প্রধান নির্বাচন কমিশন সিইসি কেএম নূরুল হুদ উদ্...
3,আওয়ামী লীগ সম্পাদক সড়ক পরিবহন সেতুমন্ত্রী ওব...
4,সোহেল তাজ আস দরজায় রেডি চলতি বছর এপ্রিলে খবরে ...
...,...
2995,নির্বাচন একদিন আদালত নির্দেশ স্থগিত ছাত্রদল কা...
2996,আর্থিক প্রতিষ্ঠান বাংলাদেশ ফাইন্যান্স লিমিটেডে...
2997,নির্বাচনে অনিয়ম দুর্নীতির অভিযোগ ঢাকা সিটি কর্...
2998,মো মনিরুজ্জামান সোমব বাংলাদেশ পানি উন্নয়ন বোর্...


In [ ]:
real = pd.concat([df_0['Text'], df_1], ignore_index=True).to_frame(name='Text')

In [ ]:
real

,Text
0,Jugantor: Bangla Newspaper | Bangla News | ব্র...
1,স্থানীয় সরকার বিশেষজ্ঞ তোফায়েল আহমেদ আর নেই
2,বুধবার (৮ অক্টোবর) সন্ধ্যা সাড়ে ৭টার দিকে রাজধ...
3,"০৮ অক্টোবর ২০২৫, ১১:২৪ পিএম"
4,জিয়াউর রহমানের মাজারে কুরআন তেলাওয়াত ও দোয়া কর...
...,...
21003,নির্বাচন একদিন আদালত নির্দেশ স্থগিত ছাত্রদল কা...
21004,আর্থিক প্রতিষ্ঠান বাংলাদেশ ফাইন্যান্স লিমিটেডে...
21005,নির্বাচনে অনিয়ম দুর্নীতির অভিযোগ ঢাকা সিটি কর্...
21006,মো মনিরুজ্জামান সোমব বাংলাদেশ পানি উন্নয়ন বোর্...


In [ ]:
fake_0 = pd.read_csv('/content/drive/MyDrive/Research/Fake news/fake_news.csv')

In [ ]:
df_2 = fake_0["headline"]
df_2

,headline
0,মুরগির হামলায় শেয়াল নিহত
1,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম
2,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...
3,অবসর নেয়ার ঘোষণা দিলেন মেসি !
4,"মাদারফাকার নহে, ব্রাদারফাকার: সাকা | দৈনিক মতি..."
...,...
1294,রাজশাহীতে স্বামীর হাঁচির আওয়াজে ঘুমের মধ্যেই স...
1295,IAS ইন্টারভিউ এ মেয়েটিকে জিজ্ঞসা করা হল মেয়েদে...
1296,শাওমি মোবাইল বিস্ফোরণে কলেজছাত্রের মৃত্যু!
1297,ফেসবুকে যেভাবে নতুন মন্ত্রীদের অভিনন্দন বার্তা...


In [ ]:
df_3 = fake_0["content"]
df_3

,content
0,"বাংলায় একটা প্রবাদ আছে, শেয়ালের কাছে মুরগী বর্..."
1,"BTV থেকে লোকজন আসছে, ইন্টারভিউ নিবে।চারজনের টি..."
2,অদ্ভুত বিরোধীদলহীনতায় ভুগছে সরকার। এ এক অন্যরক...
3,রাশিয়া বিশ্বকাপ নকআউট পর্বে ফ্রান্সের সাথে ৪-৩...
4,"নিজস্ব মতিবেদক‘মাদারফাকার নহে, আমি ব্রাদারফাকা..."
...,...
1294,রাজশাহীর সাহেব বাজার মোড়ে রাতে ঘুমের মধ্যে হাস...
1295,IAS পরীক্ষায় এমন এমন অবাক করে দেওয়া প্রশ্ন এসে...
1296,শাওমি মোবাইল ফোন বিস্ফোরণ হয়ে স্বপ্নীল মজুমদা...
1297,আপনাকে নতুন মন্ত্রিসভার সদস্যদের অভিনন্দন জানা...


In [ ]:
fake_1 = pd.read_excel('/content/drive/MyDrive/Research/Fake news/fake.xlsx')

In [ ]:
df_4 = fake_1["Review"]
df_4

,Review
0,।।।।।।।।।।।।Buy one get one Mexican পাস্তা।।।...
1,।।#review দেখে গেছিলাম\nভাবতেই অবাক লাগে ১২ রক...
2,#Partner_Platter ❤\n#বিগ_বাইট রেষ্টুরেন্ট এবার...
3,অথেনটিক পিজ্জা খেতে চাইলে এইটা মাস্ট ট্রাই ❤️\...
4,অনলি ফর ইউনিভার্সিটি কাপল স্টুডেন্ট \nধানমন্ডি...
...,...
1334,#২_টাকায়_ডিপডিশপিজ্জা_খেলাম \nসদ্য চাকরি পাওয়া...
1335,49 tk te ki pizza pawa somvob ?? \nhae somvob ...
1336,**৫৫ টাকায় জুসি বার্গার খেতে চান?🤔Burger BD ত...
1337,৫৫০টাকা তে টেবিল ভর্তি খাবার পেলাম আর জনপ্রতি ...


In [ ]:
fake = pd.concat([df_2, df_3, df_4], ignore_index=True).to_frame(name='Text')

In [ ]:
real['Label'] = "Real"
fake['Label'] = "Fake"

In [ ]:
real

,Text,Label
0,Jugantor: Bangla Newspaper | Bangla News | ব্র...,Real
1,স্থানীয় সরকার বিশেষজ্ঞ তোফায়েল আহমেদ আর নেই,Real
2,বুধবার (৮ অক্টোবর) সন্ধ্যা সাড়ে ৭টার দিকে রাজধ...,Real
3,"০৮ অক্টোবর ২০২৫, ১১:২৪ পিএম",Real
4,জিয়াউর রহমানের মাজারে কুরআন তেলাওয়াত ও দোয়া কর...,Real
...,...,...
21003,নির্বাচন একদিন আদালত নির্দেশ স্থগিত ছাত্রদল কা...,Real
21004,আর্থিক প্রতিষ্ঠান বাংলাদেশ ফাইন্যান্স লিমিটেডে...,Real
21005,নির্বাচনে অনিয়ম দুর্নীতির অভিযোগ ঢাকা সিটি কর্...,Real
21006,মো মনিরুজ্জামান সোমব বাংলাদেশ পানি উন্নয়ন বোর্...,Real


In [ ]:
fake

,Text,Label
0,মুরগির হামলায় শেয়াল নিহত,Fake
1,বিটিভিতে যেবার আমি ইন্টারভিউ দিতে গেলাম,Fake
2,বিদেশ থেকে উন্নতমানের বিরোধীদল আমদানি করার পরা...,Fake
3,অবসর নেয়ার ঘোষণা দিলেন মেসি !,Fake
4,"মাদারফাকার নহে, ব্রাদারফাকার: সাকা | দৈনিক মতি...",Fake
...,...,...
3932,#২_টাকায়_ডিপডিশপিজ্জা_খেলাম \nসদ্য চাকরি পাওয়া...,Fake
3933,49 tk te ki pizza pawa somvob ?? \nhae somvob ...,Fake
3934,**৫৫ টাকায় জুসি বার্গার খেতে চান?🤔Burger BD ত...,Fake
3935,৫৫০টাকা তে টেবিল ভর্তি খাবার পেলাম আর জনপ্রতি ...,Fake


In [ ]:
data = pd.concat([real, fake], ignore_index=True)

In [ ]:
data = data.sample(frac=1)

In [ ]:
data

,Text,Label
8224,ফেব্রুয়ারি 2022 (105),Real
15965,এমফিল লিডিং টু পিএইচডি প্রোগ্রামে ভর্তি বিজ্ঞপ...,Real
8677,নিজেই নিজেকে বিয়ে করলেন,Real
21940,দুধের শিশু বলায় ওয়াটসনকে দেখে নেয়ার হুমকি দিলে...,Fake
4206,Musk becomes first person with net worth of $5...,Real
...,...,...
5775,Germany restricts online cannabis sales,Real
6045,Galaxy Note7 recall will cost Samsung $3.9 bil...,Real
2704,অন্তর্বর্তী সরকারের সময়েই আবু সাঈদ হত্যার বিচা...,Real
20742,ইসলামী ব্যাংক বাংলাদেশ পিএলসির এজেন্ট ব্যাংকি...,Real


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, roc_curve, auc

from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
tfidf_matrix = tfidf_vectorizer.fit_transform(data['Text'])

In [ ]:
X = tfidf_matrix
y = data['Label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
rf_pred = rf_model.predict(X_test)
print(accuracy_score(y_test, rf_pred))
print(classification_report(y_test, rf_pred))

0.9619162156744838
              precision    recall  f1-score   support

        Fake       0.95      0.80      0.87       781
        Real       0.96      0.99      0.98      4208

    accuracy                           0.96      4989
   macro avg       0.96      0.89      0.92      4989
weighted avg       0.96      0.96      0.96      4989



In [ ]:
svm_model = SVC(probability=True, random_state=42)
svm_model.fit(X_train, y_train)

SVC(probability=True, random_state=42)

In [ ]:
svm_pred = svm_model.predict(X_test)
print(accuracy_score(y_test, svm_pred))
print(classification_report(y_test, svm_pred))

0.9617157747043495
              precision    recall  f1-score   support

        Fake       0.97      0.78      0.86       781
        Real       0.96      1.00      0.98      4208

    accuracy                           0.96      4989
   macro avg       0.97      0.89      0.92      4989
weighted avg       0.96      0.96      0.96      4989



In [ ]:
lr_model = LogisticRegression(random_state=42, solver='liblinear')
lr_model.fit(X_train, y_train)

LogisticRegression(random_state=42, solver='liblinear')

In [ ]:
lr_pred = lr_model.predict(X_test)
print(accuracy_score(y_test, lr_pred))
print(classification_report(y_test, lr_pred))

0.9551012226899178
              precision    recall  f1-score   support

        Fake       0.97      0.74      0.84       781
        Real       0.95      1.00      0.97      4208

    accuracy                           0.96      4989
   macro avg       0.96      0.87      0.91      4989
weighted avg       0.96      0.96      0.95      4989



In [ ]:
knn_model = KNeighborsClassifier()
knn_model.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
knn_pred = knn_model.predict(X_test)
print(accuracy_score(y_test, knn_pred))
print(classification_report(y_test, knn_pred))

0.8639005812788134
              precision    recall  f1-score   support

        Fake       0.77      0.19      0.30       781
        Real       0.87      0.99      0.92      4208

    accuracy                           0.86      4989
   macro avg       0.82      0.59      0.61      4989
weighted avg       0.85      0.86      0.83      4989



In [ ]:
gb_model = GradientBoostingClassifier(random_state=42)
gb_model.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

In [ ]:
gb_pred = gb_model.predict(X_test)
print(accuracy_score(y_test, gb_pred))
print(classification_report(y_test, gb_pred))

0.9478853477650832
              precision    recall  f1-score   support

        Fake       0.96      0.69      0.81       781
        Real       0.95      1.00      0.97      4208

    accuracy                           0.95      4989
   macro avg       0.96      0.84      0.89      4989
weighted avg       0.95      0.95      0.94      4989



In [ ]:
gnb_model = GaussianNB()
X_train_dense = X_train.toarray()
gnb_model.fit(X_train_dense, y_train)

GaussianNB()

In [ ]:
gnb_pred = gnb_model.predict(X_test.toarray())
print(accuracy_score(y_test, gnb_pred))
print(classification_report(y_test, gnb_pred))

0.6941270795750651
              precision    recall  f1-score   support

        Fake       0.33      0.93      0.49       781
        Real       0.98      0.65      0.78      4208

    accuracy                           0.69      4989
   macro avg       0.65      0.79      0.63      4989
weighted avg       0.88      0.69      0.74      4989



In [ ]:
estimators = [
    ('rf', rf_model),
    ('svm', svm_model),
    ('lr', lr_model),
    ('knn', knn_model),
    ('gb', gb_model)
]

voting_clf = VotingClassifier(estimators=estimators, voting='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('svm', SVC(probability=True, random_state=42)),
                             ('lr',
                              LogisticRegression(random_state=42,
                                                 solver='liblinear')),
                             ('knn', KNeighborsClassifier()),
                             ('gb',
                              GradientBoostingClassifier(random_state=42))])

In [ ]:
voting_pred = voting_clf.predict(X_test)
print(accuracy_score(y_test, voting_pred))
print(classification_report(y_test, voting_pred))

0.9571056323912608
              precision    recall  f1-score   support

        Fake       0.98      0.74      0.84       781
        Real       0.95      1.00      0.98      4208

    accuracy                           0.96      4989
   macro avg       0.97      0.87      0.91      4989
weighted avg       0.96      0.96      0.95      4989

